<a href="https://colab.research.google.com/github/jpo85/meshroom_colab/blob/main/MeshroomColab_Ver1.3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fotogrammetria tramite google Cloud & Meshroom

Questa pagina web si prefigge lo scopo di aiutare l'elaborazione di un dataset di foto per tentare una prima elaborazione e ricostruzione fotogrammetrica. Gli strumenti che vengono utilizzati per eseguire l'elaborazione sono il servizio Google colab e meshroom.

Il servizio Google Colab dove è anche ospitata la seguente pagina web ha lo scopo di eseguire in cloud gli script shell e python per l'elaborazione delle immagini e la creazione della mesh 3d finale.
Google Colab è una piattaforma di google nata principalmente per eseguire quelle operazioni di machine learning che richiedono prestazioni e potenza di calcolo elevata. Nel nostro caso specifico vengono messe a disposizione dall'infrastruttura cloud di google le seguenti potenze di calcolo:

*   32 GB di RAM
*   GPU con 16GB di RAM dedicata e funzionalità CUDA
*   Circa 40GB su filesystem

Meshroom è un software con licenza opensource che consente tramite foto/video di effettuare una ricostruzione fotogrammetrica basandosi sul framework AliceVision. L'interfaccia di utilizzo è basata sulla logica dei workflow in cui ogni blocco è collegato al successivo e salva su una folder l'elaborazione temporanea. Questo processo consente di creare anche workflow custom in cui è possibile avere più elaborazione in formato diverso.





# Come funziona

la logica di questa guida è innanzitutto capire che non si tratta di una semplice pagina web statica ma di una vera e propria guida stepbystep dove dopo ogni spiegazione è presente direttamente il codice gà scritto da eseguire. Su ogni blocco è già scritto tutto il codice necessario per una elaborazione standard. Il workflow standard che viene eseguito in questa guida consente di prendere una dataset di foto e generare un file obj 3d con le sue relative texture.
il risutato finale sarà:

*   1 file in formato .obj
*   vari file di texture in formato .jpeg
*   1 file con la libreria dei materia in formato .mtl

Il codice presente è aperto perciò è modificabile a seconda delle proprie esigenze. Ad esempio se si dispone di un workflow meshroom custom si può inserire il comando per l'importazione del workflow specifico, e far eseguire quello anzichè quello di default del software.






# Prerequisiti

Ll codice per essere eseguito prevede una minima configurazione iniziale e non serve successivamente scrivere codice.
Come prerequisito serve avere un'account google drive con spazio a sufficienza per fare l'upload delle foto e salvare la mesh finale. Lo script infatti utilizza come input e output due folder di google drive che dovranno essere create a mano per consentire al codice di prendere le foto, elaborarle e salvare la mesh finale.

# Esecuzione stepbystep

Di seguito sono riportati una serie di passaggi numerati da eseguire uno dopo l'altro che guideranno l'utente nell'elaborazione fotogrammetrica.

**0. Configura il runtime in modalità GPU e fai il check**

questo blocco una volta eseguito consente di verificare se il runtime è configurato per eseguire l'elaborazione tramite l'utilizzo della GPU. Nel caso in cui l'esito fosse negativo, si riporta di seguito come abilitare tale funzione. Si ricorda che la GPU è fondamentale per una esecuzione più veloce dello script

abilitare le GPU per il notebook:

Vai a Modifica → Impostazioni blocco note
Seleziona GPU dal menu a discesa Acceleratore hardware
Successivamente, confermeremo che possiamo connetterci alla GPU con tensorflow

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

**1. connetti a Google Drive**

connetti a google drive utilizzando l'authorization code


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

**2. crea le folder necessarie e fa il download di meshroom:**
*   /content - contiene la root del pacchetto meshroom che sarà eseguito
*   /content/temp - contiene il pacchetto meshroom scaricato
*   /content/meshroom - contiene tutti gli eseguibili di meshroom scompattati e pronti per essere eseguiti








In [ ]:
%cd /content
!mkdir temp
!mkdir meshroom
%cd temp
!wget -N https://github.com/alicevision/meshroom/releases/download/v2019.2.0/Meshroom-2019.2.0-linux.tar.gz
!tar -xvf Meshroom-2019.2.0-linux.tar.gz -C ../meshroom


**3. esegui meshroom**
*   il nodi salvano temporaneamente nella folder **/tmp/MeshroomCache**
*   il file obj finale viene salvato nella folder **/content/drive/output**


In [ ]:
!/content/meshroom/Meshroom-2019.2.0/meshroom_photogrammetry --input '/content/drive/MyDrive/input' --output '/content/drive/MyDrive/output'

**4. Preview Mesh usando Trimesh** 

ancora in fase sperimentale

In [ ]:
!pip install numpy
!pip install trimesh
%cd /content/drive/output

lancia preview

In [ ]:
import numpy as np
import trimesh
mesh = trimesh.load_mesh('texturedMesh.obj')
mesh.show()